---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

* 워크플로우
  * `HubClient() 객체` 생성 → `client.push()` 사용

---

In [1]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

---

#### **6) `Chain of Density` 요약 (한국어)**

* 역할
  * 긴 기사나 글(`ARTICLE`)을 **점점 더 짧고 밀도 높게 요약** 하는 과정을 **5단계 반복**
  * 매 단계마다 이전 요약에 **빠진 중요한 `엔티티(정보 단위)`를 찾아서 추가** 하고 **다시 요약**

* 특징
  * 첫 요약은 좀 길고 덜 구체적, 이후 점점 압축하면서도 중요한 내용 빠짐 없이 포함
  * 요약 길이 `일정 유지`
  * 최종 결과는 `JSON 리스트`로 단계별 요약 결과 제공

In [3]:
# Chain of Density 요약 (한국어)

# 프롬프트의 제목
# 특별한 기능 X, 프롬프트 식별하기 위해 사용 (문서의 내용을 지도처럼 요약한다는 의미)
prompt_title = "chain-of-density-korean"

# 프롬프트의 본문
prompt_template = """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 5 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (100 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer in JSON. The JSON should be a list (length 5) of dictionaries whose keys are "Missing_Entities" and "Denser_Summary".
Use only KOREAN language to reply."""

prompt_density = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
# 허브에 올리기 
result_density = client.push_prompt("prompt_template_density", object=prompt_density)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_density)
print('\n', "-" * 50)

# 허브에서 가져오기 
prompt_density_from_hub = client.pull_prompt("prompt_template_density")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")

# 최근 커밋 해시 조회하기
prompt_id = "prompt_template_density"
prompt_info_density = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_density)

prompt_info_density = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_density.__dict__.items():
    print(f"{key}: {value}")

<small>

* 셀 출력

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_density/2d444953?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.
    프롬프트 정보: repo_handle='prompt_template_density' description='' readme='' id='34a6980b-a093-4615-9080-5b77aa09de90' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 14, 1, 20, 18, 177679) updated_at=datetime.datetime(2025, 8, 14, 1, 20, 20, 155489) is_public=False is_archived=False tags=['ChatPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_density' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='2d444953a17b35545cfb2b583ea15e10c1254a398a0c1f7124075209358ad52f' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_density
    description: 
    readme: 
    id: 34a6980b-a093-4615-9080-5b77aa09de90
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-14 01:20:18.177679
    updated_at: 2025-08-14 01:20:20.155489
    is_public: False
    is_archived: False
    tags: ['ChatPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_density
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: 2d444953a17b35545cfb2b583ea15e10c1254a398a0c1f7124075209358ad52f
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None
    ```
    

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc               # 위에서 정의함

# chain_map 구성
chain_density = prompt_density | gemini_lc | StrOutputParser()

# context_text 입력하기
# 인공지능과 교육의 융합: 미래 교실의 변화
context_text = """
최근 인공지능(AI) 기술이 교육 분야에 접목되면서 학습 방식에 근본적인 변화가 일어나고 있습니다. 많은 교육 기관들이 AI 기반 맞춤형 학습 플랫폼을 도입하여 학생 개개인의 학습 속도와 수준에 맞는 콘텐츠를 제공하고 있습니다. 예를 들어, AI 튜터는 학생의 취약점을 정확히 파악하여 보충 학습 자료를 추천하고, 질문에 대한 즉각적인 피드백을 제공함으로써 학습 효율을 극대화합니다.

이러한 변화는 교사의 역할에도 영향을 미치고 있습니다. AI가 반복적이고 단순한 행정 업무(예: 채점, 출석 관리)를 대신하면서, 교사들은 학생들과의 상호작용, 창의적인 문제 해결 능력 향상 등 보다 질 높은 교육 활동에 집중할 수 있게 되었습니다. 또한, AI는 교사들에게 학생들의 학습 데이터를 분석한 심층적인 통찰을 제공하여, 맞춤형 교육 계획을 수립하는 데 도움을 줍니다.

하지만 AI 교육의 확산에는 여러 과제가 남아 있습니다. 고가의 AI 솔루션 도입 비용은 학교 예산에 큰 부담이 될 수 있으며, 모든 학생이 디지털 기기에 접근할 수 있는 환경을 조성하는 것도 중요한 문제입니다. 더불어, AI가 생성한 데이터의 개인정보 보호 문제와 AI 기술에 대한 교사들의 전문성 부족 또한 해결해야 할 주요 이슈로 꼽힙니다. 그럼에도 불구하고, 전문가들은 AI가 교육 불평등을 해소하고 미래 사회에 필요한 인재를 양성하는 데 필수적인 도구가 될 것이라고 전망하고 있습니다.
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(7.4s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "AI 기반 맞춤형 학습 플랫폼; AI 튜터; 학생들의 취약점 파악 및 보충 학습 자료 추천",
        "Denser_Summary": "본 글은 최근 교육 분야에 인공지능(AI) 기술이 접목되면서 발생하는 근본적인 변화에 대해 논하고 있습니다. 특히, 많은 교육 기관들이 AI 기반 맞춤형 학습 플랫폼을 도입하여 학생 개개인의 학습 속도와 수준에 맞는 콘텐츠를 제공하는 추세를 소개하고 있습니다. 이러한 맥락에서 AI 튜터는 학생들의 취약점을 정확히 파악하고 이에 맞는 보충 학습 자료를 추천함으로써 학습 효율을 극대화하는 데 기여하고 있습니다. 또한, AI는 학생들의 질문에 대한 즉각적인 피드백을 제공하는 기능도 수행합니다. 이처럼 AI 기술은 교육 현장에 새로운 가능성을 제시하고 있으며, 앞으로의 발전이 기대됩니다."
    },
    {
        "Missing_Entities": "교사의 역할 변화; 반복적/단순 행정 업무(채점, 출석 관리) AI 대체; 질 높은 교육 활동 집중",
        "Denser_Summary": "교육 분야에 AI 기술이 접목되며 학습 방식에 근본적 변화가 일어나고 있습니다. AI 기반 맞춤형 학습 플랫폼 도입으로 학생 개개인에게 맞는 콘텐츠 제공이 가능해졌으며, AI 튜터는 학생 취약점 파악 및 보충 자료 추천, 즉각적 피드백으로 학습 효율을 높입니다. 이로 인해 교사들은 채점, 출석 관리 등 반복적/단순 행정 업무를 AI에 위임하고, 학생들과의 상호작용 및 창의적 문제 해결 능력 향상 등 질 높은 교육 활동에 집중할 수 있게 되었습니다. 이러한 변화는 교육의 질적 향상을 기대하게 합니다."
    },
    {
        "Missing_Entities": "AI 기반 교사 지원; 학습 데이터 분석 심층 통찰 제공; 맞춤형 교육 계획 수립 지원",
        "Denser_Summary": "교육 분야 AI 접목은 학습 방식 근본적 변화를 가져옵니다. AI 맞춤형 학습 플랫폼은 학생별 학습 속도/수준에 맞는 콘텐츠를 제공하며, AI 튜터는 학생 취약점 파악, 보충 자료 추천, 즉각적 피드백으로 학습 효율을 극대화합니다. 교사 역할도 변화하여, AI가 채점, 출석 관리 등 단순 업무를 대체함으로써 교사는 학생 상호작용, 창의력 향상 등 질 높은 교육에 집중하게 됩니다. 또한 AI는 학습 데이터 분석을 통한 심층 통찰을 교사에게 제공하여 맞춤형 교육 계획 수립을 지원합니다."
    },
    {
        "Missing_Entities": "AI 교육 확산 과제; 고가 AI 솔루션 도입 비용 부담; 디지털 기기 접근 환경 조성 문제",
        "Denser_Summary": "교육 분야 AI 도입은 학습 방식 근본적 변화를 이끌고 있습니다. AI 맞춤형 학습 플랫폼은 학생별 맞춤 콘텐츠를 제공하며, AI 튜터는 취약점 파악, 보충 자료 추천, 즉각적 피드백으로 학습 효율을 높입니다. 교사는 단순 업무를 AI에 위임, 학생과의 상호작용 및 창의력 향상 등 질 높은 교육에 집중하고, AI는 학습 데이터 분석 통찰로 맞춤형 교육 계획 수립을 지원합니다. 그러나 AI 교육 확산에는 고가 솔루션 도입 비용 부담, 디지털 기기 접근 환경 조성 문제가 과제로 남아있습니다."
    },
    {
        "Missing_Entities": "개인정보 보호 문제; AI 기술 전문성 부족; 교육 불평등 해소 및 미래 인재 양성 필수 도구 전망",
        "Denser_Summary": "교육 분야 AI는 학습 방식의 근본적 변화를 주도합니다. AI 맞춤형 학습 플랫폼은 학생별 맞춤 콘텐츠를 제공하고, AI 튜터는 취약점 파악, 보충 자료 추천, 즉각적 피드백으로 학습 효율을 극대화합니다. 교사는 단순 업무를 AI에 위임하여 학생 상호작용 및 창의력 향상 등 질 높은 교육에 집중하며, AI는 학습 데이터 분석을 통해 맞춤형 교육 계획 수립을 지원합니다. 그러나 고가 솔루션 비용, 디지털 기기 접근성, 개인정보 보호, 교사 전문성 부족 등이 AI 교육 확산의 과제입니다. 그럼에도 AI는 교육 불평등 해소 및 미래 인재 양성에 필수적 도구로 전망됩니다."
    }
    ]
    ```

In [ ]:
# context_text 입력하기_2
# 영어 기사, 비슷한 분량, 다른 주제의 기사로 테스트
# Culture & Arts News Article: The Ancient City and Modern Conflicts
context_text = """
The historic city of Petra, a UNESCO World Heritage site, is at the heart of a heated debate between conservationists and local business owners. 
Known for its rock-cut architecture and intricate water conduit system, the ancient city attracts millions of tourists annually, serving as a vital economic engine for the region.
However, this massive influx of visitors is taking a heavy toll on the delicate sandstone structures. 
Conservationists are warning that foot traffic, human contact, and the vibrations from nearby construction are accelerating erosion and causing irreversible damage to the iconic Treasury and Monastery facades.
They advocate for stricter regulations on tourist access, including limiting visitor numbers and restricting certain areas.
Conversely, many local merchants and tour operators argue that such restrictions would severely harm their livelihoods, which depend entirely on tourism.
They propose alternative solutions, such as developing new visitor routes and creating artificial lighting to showcase the sites at night, to balance preservation with economic growth.
The ongoing conflict highlights the global challenge of managing cultural heritage sites in a sustainable manner for future generations.
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(5.7s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "페트라; 유네스코 세계 문화 유산; 암각 건축; 수로 시스템",
        "Denser_Summary": "이 글은 역사적인 도시 페트라에 대한 논쟁을 다룹니다. 페트라는 수백만 명의 관광객을 끌어들이는 중요한 경제적 동력이지만, 보존주의자들은 관광객의 발길과 접촉, 건설 진동이 섬세한 사암 구조물에 가속화된 침식과 돌이킬 수 없는 손상을 일으킨다고 경고하고 있습니다. 이에 따라 방문객 수 제한과 특정 구역 통제와 같은 엄격한 규제를 제안하고 있습니다. 반면, 지역 상인과 여행사들은 이러한 제한이 자신들의 생계에 심각한 타격을 줄 것이라고 주장하며, 새로운 방문 경로 개발 및 야간 조명 설치와 같은 대안을 제시하며 보존과 경제 성장 간의 균형을 맞추고자 합니다. 이러한 갈등은 미래 세대를 위한 문화 유산지의 지속 가능한 관리라는 전 세계적인 과제를 조명합니다."
    },
    {
        "Missing_Entities": "재무부; 수도원; 발걸음; 인간 접촉; 진동; 건설",
        "Denser_Summary": "이 글은 유네스코 세계 문화 유산인 페트라에서 보존주의자와 지역 사업주 간의 첨예한 논쟁을 조명합니다. 암각 건축과 정교한 수로 시스템으로 유명한 이 고대 도시는 매년 수백만 명의 관광객을 유치하며 지역 경제의 핵심 동력 역할을 합니다. 그러나 이러한 막대한 방문객 유입은 섬세한 사암 구조물에 큰 부담을 주고 있습니다. 보존주의자들은 방문객의 발걸음, 인간 접촉, 그리고 근처 건설 현장의 진동이 재무부와 수도원 정면의 침식을 가속화하고 돌이킬 수 없는 손상을 유발한다고 경고하며, 방문객 수 제한 및 특정 구역 통제와 같은 엄격한 규제를 옹호합니다. 지역 상인과 여행사들은 이러한 제한이 생계에 타격을 줄 것이라며, 새로운 방문 경로 개발 및 야간 조명 설치로 보존과 경제 성장의 균형을 이루자고 제안합니다."
    },
    {
        "Missing_Entities": "관광객 수 제한; 특정 구역 제한; 지역 상인; 여행사; 생계; 새로운 방문 경로; 야간 조명",
        "Denser_Summary": "유네스코 세계 문화 유산인 페트라는 보존주의자와 지역 사업주 간의 치열한 논쟁의 중심에 서 있습니다. 암각 건축과 정교한 수로 시스템으로 유명한 페트라는 연간 수백만 관광객을 유치하며 지역 경제를 견인합니다. 그러나 과도한 방문객은 섬세한 사암 구조물에 발걸음, 인간 접촉, 건설 진동으로 인한 침식과 돌이킬 수 없는 손상을 가속화합니다. 보존주의자들은 관광객 수 제한 및 특정 구역 제한을 주장하는 반면, 지역 상인과 여행사는 생계가 위협받는다며 새로운 방문 경로 개발 및 야간 조명 설치를 통한 보존과 경제 성장 균형을 제안합니다. 이 갈등은 문화 유산지의 지속 가능한 관리에 대한 전 세계적 도전을 보여줍니다."
    },
    {
        "Missing_Entities": "문화 유산지; 지속 가능한 관리; 미래 세대; 경제 성장; 보존",
        "Denser_Summary": "페트라, 유네스코 세계 문화 유산지는 보존주의자와 지역 사업주 간의 논쟁의 중심입니다. 암각 건축과 수로 시스템으로 유명한 이곳은 연간 수백만 관광객을 유치하며 지역 경제를 활성화하지만, 발걸음, 접촉, 건설 진동은 사암 구조물에 침식과 손상을 가속화합니다. 보존주의자는 관광객 수 제한 및 구역 제한을 주장하나, 지역 상인과 여행사는 생계 문제로 새로운 방문 경로 개발, 야간 조명 설치 등 보존과 경제 성장 균형을 제안합니다. 이 갈등은 문화 유산지의 지속 가능한 관리라는 전 세계적 과제를 부각합니다."
    },
    {
        "Missing_Entities": "암석 절단 건축; 수로 시스템; 경제적 엔진; 사암 구조물; 재정적 타격; 대안적 해결책",
        "Denser_Summary": "페트라, 유네스코 세계 문화 유산지는 보존주의자와 지역 사업주 간의 논쟁의 중심입니다. 암석 절단 건축과 정교한 수로 시스템으로 유명한 이곳은 연간 수백만 관광객을 유치하며 지역 경제의 중요한 엔진 역할을 합니다. 하지만 과도한 방문객은 섬세한 사암 구조물에 발걸음, 접촉, 건설 진동으로 인한 침식을 가속화합니다. 보존주의자는 방문객 수 제한 및 특정 구역 제한을 주장하지만, 지역 상인과 여행사는 생계에 재정적 타격을 우려하며 새로운 방문 경로 개발, 야간 조명 설치와 같은 대안적 해결책을 통해 보존과 경제 성장의 균형을 모색합니다."
    }
    ]
    ```

In [ ]:
# context_text 입력하기_3
# 다른 언어 기사(스페인어), 비슷한 분량, 다른 주제의 기사로 테스트
# Spanish Sports News Article: The Future of Football
context_text = """
El fútbol mundial se encuentra en una encrucijada, con la tecnología transformando no solo el juego en sí, sino también la forma en que los clubes gestionan a sus jugadores y finanzas. 
La reciente transferencia de la joven estrella, Carlos Ruiz, por una cifra récord de 150 millones de euros a un club de la Premier League, ha puesto de relieve la escalada de precios en el mercado.
Los equipos de élite ahora dependen en gran medida de los análisis de datos avanzados para identificar talentos emergentes y evaluar el rendimiento de los jugadores.
Además, la implementación del VAR (Video Assistant Referee) ha generado debates continuos sobre la equidad y la fluidez del juego, a pesar de su objetivo de reducir los errores arbitrales.
Los expertos predicen que la próxima década verá una mayor integración de la realidad virtual en la experiencia de los aficionados y una monetización más sofisticada a través de los e-sports.
Este panorama en constante evolución desafía a las ligas y a los equipos a adaptarse rápidamente para mantenerse competitivos.
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(4.5s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "Carlos Ruiz; 150 millones de euros; Premier League",
        "Denser_Summary": "이 기사는 축구계가 기술 혁신으로 인해 중요한 전환점에 서 있음을 논하고 있습니다. 특히, 게임 자체뿐만 아니라 클럽들이 선수들과 재정을 관리하는 방식에도 상당한 변화가 일어나고 있음을 강조합니다. 이러한 변화는 선수 이적 시장의 가격 상승을 부추기고 있으며, 이는 최근 젊은 스타 플레이어인 카를로스 루이스의 1억 5천만 유로라는 기록적인 이적료를 통해 명확히 드러납니다. 또한, 엘리트 팀들은 재능 있는 선수들을 발굴하고 선수들의 경기력을 평가하는 데 있어 고급 데이터 분석에 크게 의존하고 있습니다. 이러한 기술적 발전은 축구 산업 전반에 걸쳐 새로운 기회와 도전을 제시하고 있습니다."
    },
    {
        "Missing_Entities": "VAR; e-sports",
        "Denser_Summary": "축구계는 기술 혁신으로 중대한 전환점을 맞이하며, 게임 자체와 클럽의 선수 및 재정 관리에 변화를 가져오고 있습니다. 젊은 스타 카를로스 루이스의 1억 5천만 유로 프리미어 리그 이적은 시장 가격 급등을 보여줍니다. 엘리트 팀은 데이터 분석으로 신예 발굴 및 선수 평가에 의존합니다. VAR 도입은 오심 감소 목표에도 불구하고 공정성과 경기 흐름에 대한 논쟁을 지속시킵니다. 전문가들은 향후 10년간 가상현실 통합과 e-스포츠를 통한 수익화가 확대될 것으로 예측하며, 이러한 환경 변화는 리그와 팀들에게 신속한 적응을 요구합니다."
    },
    {
        "Missing_Entities": "데이터 분석; 가상현실",
        "Denser_Summary": "축구계는 기술 혁신으로 중대한 전환점을 맞이하며, 게임 자체와 클럽의 선수 및 재정 관리에 변화를 가져오고 있습니다. 젊은 스타 카를로스 루이스의 1억 5천만 유로 프리미어 리그 이적은 시장 가격 급등을 보여줍니다. 엘리트 팀은 고급 데이터 분석으로 신예 발굴 및 선수 평가에 의존합니다. VAR 도입은 오심 감소 목표에도 불구하고 공정성과 경기 흐름에 대한 논쟁을 지속시킵니다. 전문가들은 향후 10년간 가상현실 통합과 e-스포츠를 통한 수익화가 확대될 것으로 예측하며, 이러한 환경 변화는 리그와 팀들에게 신속한 적응을 요구합니다."
    },
    {
        "Missing_Entities": "공정성; 유동성",
        "Denser_Summary": "축구계는 기술 혁신으로 중대한 전환점을 맞이하며, 게임 자체와 클럽의 선수 및 재정 관리에 변화를 가져오고 있습니다. 젊은 스타 카를로스 루이스의 1억 5천만 유로 프리미어 리그 이적은 시장 가격 급등을 보여줍니다. 엘리트 팀은 고급 데이터 분석으로 신예 발굴 및 선수 평가에 의존합니다. VAR 도입은 오심 감소 목표에도 불구하고 공정성과 경기 유동성에 대한 논쟁을 지속시킵니다. 전문가들은 향후 10년간 가상현실 통합과 e-스포츠를 통한 수익화가 확대될 것으로 예측하며, 이러한 환경 변화는 리그와 팀들에게 신속한 적응을 요구합니다."
    },
    {
        "Missing_Entities": "재정 관리; 오심 감소",
        "Denser_Summary": "축구계는 기술 혁신으로 중대한 전환점을 맞이하며, 게임 자체와 클럽의 선수 및 재정 관리에 변화를 가져오고 있습니다. 젊은 스타 카를로스 루이스의 1억 5천만 유로 프리미어 리그 이적은 시장 가격 급등을 보여줍니다. 엘리트 팀은 고급 데이터 분석으로 신예 발굴 및 선수 평가에 의존합니다. VAR 도입은 오심 감소 목표에도 불구하고 공정성과 경기 유동성에 대한 논쟁을 지속시킵니다. 전문가들은 향후 10년간 가상현실 통합과 e-스포츠를 통한 수익화가 확대될 것으로 예측하며, 이러한 환경 변화는 리그와 팀들에게 신속한 적응을 요구합니다."
    }
    ]
    ```

---

<small>

##### **`토큰 단위`** = **처리 속도** 에 중요한 영향

* 처리 속도 차이에 대한 설명 = `토큰화(Tokenization)` 과정에서 발생
  * **토큰화**: 언어 모델은 텍스트를 단어가 아닌 '토큰'이라는 단위로 쪼개서 처리
  * **영어**: 영어는 **알파벳** 과 **띄어쓰기** → **한 단어가 하나의 토큰으로 처리** 되는 경우가 많아 **효율적**
  * **`한국어`**
    * 한국어는 **`글자`** or **`형태소`** 단위로 토큰화되는 경우 많음
    * **`안녕하세요`** = 한 단어가 **여러 토큰** 으로 쪼개질 수 있음
    * 따라서 겉보기에는 분량이 비슷해도, **한국어 텍스트가 영어 텍스트보다 훨씬 더 많은 토큰** 으로 이루어져 있어 처리 시간이 더 오래 걸릴 수 있음

  * 또한, 모델의 학습 데이터가 영어 중심이었던 역사적 배경도 영향을 미침
  
* ---

* 토큰화가 어려운 언어
  * **`일본어`** , **`중국어`**
    * 한국어보다 **토큰 단위가 더 크거나** **토큰화가 복잡** 하여 처리 속도가 느릴 수 있음
    * 단어 사이에 **띄어쓰기가 없고** , 표의 문자인 **한자** 와 음절 문자인 **히라가나/가타카나** 를 섞어 사용하기 때문에 **토큰화 과정이 매우 복잡**
  
* ---

---

In [ ]:
# context_text 입력하기_4
# 토큰화가 복잡한 언어: 일본어로 테스트
# 비슷한 분량, 다른 주제의 기사로 테스트
# Japanese News Article: Art and Digital Transformation
context_text = """
近年、アートの世界でもデジタルトランスフォーメーションが急速に進んでいます。美術館はバーチャルツアーやオンライン展示を導入し、物理的な距離を超えて多くの人々に作品を届けるようになりました。これにより、世界中の人々が自宅にいながらにして、ルーヴル美術館やメトロポリタン美術館の貴重なコレクションを鑑賞できるようになっています。また、ブロックチェーン技術を活用したNFT（非代替性トークン）アートの登場は、デジタルアートの所有権を明確にし、新たな市場を創出しました。若手アーティストたちは、デジタルツールを駆사してこれまでにない表現方法を追求しており、既存のアートの定義を広げています。しかし、テクノロジーの進化に伴い、著作権の問題やデジタル格差といった課題も浮上しており、アートコミュニティは新しい時代の変化にどう対応していくか模索しています。
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(4.7s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "데지탈 트랜스포메이션; 가상 투어; 온라인 전시",
        "Denser_Summary": "이 글은 미술계에서 데지탈 트랜스포메이션이 급속도로 진행되고 있음을 논하고 있습니다. 많은 미술관들이 물리적인 거리감을 극복하고 더 많은 사람들에게 작품을 전달하기 위해 가상 투어와 온라인 전시를 도입하고 있습니다. 이러한 변화를 통해 전 세계의 사람들이 자신의 집에서 편안하게 세계적인 명문 미술관들의 귀중한 컬렉션을 감상할 수 있게 되었습니다. 이는 기술 발전이 예술 접근성을 어떻게 향상시키고 있는지 보여주는 중요한 사례입니다. 이로 인해 예술을 경험하는 방식에 근본적인 변화가 일어나고 있습니다."
    },
    {
        "Missing_Entities": "루브르 미술관; 메트로폴리탄 미술관; 블록체인 기술",
        "Denser_Summary": "미술계의 데지탈 트랜스포메이션은 가상 투어 및 온라인 전시 도입으로 물리적 거리를 초월해 작품 접근성을 높였습니다. 이제 루브르 미술관, 메트로폴리탄 미술관 등 세계 유수 미술관의 귀중한 컬렉션을 집에서 감상할 수 있습니다. 또한, 블록체인 기술은 NFT 아트의 등장으로 이어져 디지털 아트의 소유권을 명확히 하고 새로운 시장을 창출하는 혁신을 가져왔습니다. 이는 예술의 물리적 제약을 극복하고 새로운 형태의 가치를 부여하는 중요한 발전입니다."
    },
    {
        "Missing_Entities": "NFT (비대체성 토큰); 디지털 아트; 소유권",
        "Denser_Summary": "미술계의 데지탈 트랜스포메이션은 가상 투어 및 온라인 전시로 물리적 거리 제약을 극복, 루브르 미술관, 메트로폴리탄 미술관 컬렉션의 접근성을 높였습니다. 블록체인 기술 기반 NFT(비대체성 토큰) 아트는 디지털 아트의 소유권을 명확히 하고 새로운 시장을 창출했습니다. 이러한 기술 발전은 예술의 물리적 한계를 넘어서며, 디지털 매체를 통한 새로운 예술 경험과 거래 방식을 가능하게 했습니다. 이는 예술의 본질과 유통 방식에 대한 근본적인 변화를 시사합니다."
    },
    {
        "Missing_Entities": "젊은 예술가; 디지털 도구; 표현 방법",
        "Denser_Summary": "미술계의 데지탈 트랜스포메이션은 가상 투어 및 온라인 전시로 루브르 미술관, 메트로폴리탄 미술관 컬렉션 접근성을 높였습니다. 블록체인 기반 NFT 아트는 디지털 아트 소유권을 명확히 하고 새 시장을 창출했습니다. 젊은 예술가들은 디지털 도구를 활용해 혁신적인 표현 방법을 추구하며 기존 예술의 정의를 확장하고 있습니다. 이는 기술이 예술 창작과 향유 방식에 미치는 지대한 영향을 보여주며, 예술의 새로운 지평을 열고 있습니다. 예술은 이제 물리적 공간을 넘어선 디지털 영역으로 확장되고 있습니다."
    },
    {
        "Missing_Entities": "저작권 문제; 디지털 격차; 예술 커뮤니티",
        "Denser_Summary": "미술계 데지탈 트랜스포메이션은 가상 투어, 온라인 전시로 루브르, 메트로폴리탄 미술관 컬렉션 접근성을 높였습니다. 블록체인 기반 NFT 아트는 디지털 아트 소유권을 명확히 하고 새 시장을 창출했습니다. 젊은 예술가들은 디지털 도구로 혁신적 표현을 추구하며 예술 정의를 확장합니다. 그러나 저작권 문제, 디지털 격차 등 새로운 과제도 부상했으며, 예술 커뮤니티는 변화하는 시대에 대한 대응 방안을 모색 중입니다. 이는 기술 발전이 가져온 기회와 도전을 동시에 안고 있음을 보여줍니다."
    }
    ]
    ```

  * 결과가 빠르게 나온 이유?
    * **`모델의 최적화`**:
      * `Gemini 2.5 Flash Lite`와 같은 최신 모델들은 특정 언어에 대한 처리 효율을 높이는 최적화가 이루어져 있을 가능성
      * `한국어`, `일본어`, `중국어` 같은 아시아 언어에 대한 토큰화 및 처리 방식이 초기 모델보다 훨씬 발전했을 가능성 높음
  
    * **`토큰화의 효율`**: 
      * 일본어는 한자와 히라가나, 가타카나가 섞여 있어 복잡해 보이지만, **`모델의 토크나이저`** 가 입력된 텍스트를 **`효율적`** 으로 **`청크(chunk)`** 단위로 묶었을 수 있음
      * 사용하신 기사의 내용이 모델이 이미 잘 학습하고 효율적으로 처리할 수 있는 패턴이었을 가능성이 있음

    * **`문장 구조와 정보 밀도`**: 
      * **`텍스트의 문장 구조`** 나 **`정보 밀도 자체`** 도 영향을 미침
      * 어떤 언어의 문장이 다른 언어보다 더 많은 토큰을 요구하더라도, **`문장 내의 관계`** 나 **`정보가 더 단순`** 하면 **모델이 더 빠르게 이해하고 처리** 가능

  * 따라서 단순히 `토큰 개수가 많다/적다`로만 속도를 단정 짓기보다는, `모델의 언어별 최적화 수준`, `토큰화 방식`, 그리고 `입력된 텍스트의 특성` 등이 모두 영향을 미침

In [ ]:
# context_text 입력하기_5
# 토큰화가 복잡한 언어: 일본어로 테스트
# 문장이 담고 있는 정보의 복잡성과 깊이 높여보기
# Japanese News Article: Lifestyle and Social Trends
context_text = """
近年、日本の都市部を中心に「ミニマリスト」と呼ばれる生活様式が若者世代の間で広まっています。これは、物質的な所有物を最小限に抑え、精神的な豊かさや時間を重視するライフスタイルです。彼らは不要なモノを持たないことで、日々の選択肢を減らし、より本質的なことに集中できると主張しています。この動きは、オンラインコミュニティやSNSを通じて急速に拡散され、シンプルで持続可能な消費を促す新たな社会現象となっています。このトレンドの背景には、過剰な消費文化への反감と、不安定な経済状況の中で将来に対する不安を減ら려는 심리가影響していると考えられます。また、デジタル技術の普及も一役買っており、紙の書類や物理的なメディアをクラウドに保存することで、モノを持たない生活をさらに 용이하게しました。専門家は、このような価値観の変化が、今後の住宅、ファッション、さらには公共サービスのデザインにも影響を与えるだろうと予測しています。
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(6.6s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "ミニマリスト;物質的な所有物;精神的な豊かさ;時間を重視;過剰な消費文化への反感;不安定な経済状況;将来への不安;デジタル技術;クラウド;住宅;ファッション;公共サービス",
        "Denser_Summary": "최근 일본 도시 지역을 중심으로 '미니멀리스트'라는 생활 양식이 젊은 세대 사이에서 확산되고 있습니다. 이는 물질적인 소유물을 최소화하고 정신적인 풍요로움과 시간을 중시하는 라이프스타일을 의미합니다. 불필요한 물건을 갖지 않음으로써 일상의 선택지를 줄이고 본질적인 것에 집중할 수 있다고 그들은 주장합니다. 이러한 움직임은 온라인 커뮤니티와 SNS를 통해 빠르게 퍼져나가며, 단순하고 지속 가능한 소비를 촉진하는 새로운 사회 현상이 되었습니다. 이 트렌드의 배경에는 과도한 소비 문화에 대한 반감과 불안정한 경제 상황 속에서 미래에 대한 불안감을 줄이려는 심리가 작용하고 있는 것으로 보입니다. 또한, 디지털 기술의 보급도 한몫을 하고 있는데, 종이 서류나 물리적 미디어를 클라우드에 저장함으로써 소유하지 않는 삶을 더욱 용이하게 만들었습니다. 전문가들은 이러한 가치관의 변화가 향후 주택, 패션, 그리고 공공 서비스 디자인에도 영향을 미칠 것이라고 예측하고 있습니다. 이 현상은 물질주의에서 벗어나 삶의 질을 향상시키려는 현대 사회의 중요한 흐름을 보여줍니다."
    },
    {
        "Missing_Entities": "若者世代;本質的なこと;選択肢を減らし;シンプルで持続可能な消費; 용이하게",
        "Denser_Summary": "일본 도시의 젊은 세대 사이에서 '미니멀리스트' 생활 방식이 확산 중입니다. 이는 물질적 소유를 최소화하고 정신적 풍요와 시간을 중시하며, 불필요한 물건을 버려 선택지를 줄이고 본질에 집중하는 라이프스타일입니다. 온라인 커뮤니티와 SNS를 통해 퍼진 이 트렌드는 단순하고 지속 가능한 소비를 장려하며, 과잉 소비 문화에 대한 반감과 경제 불안, 미래 불안 해소 심리가 배경에 있습니다. 디지털 기술, 특히 클라우드 저장은 물리적 매체 없이 생활을 용이하게 했고, 이러한 가치관 변화는 주택, 패션, 공공 서비스 디자인에도 영향을 미칠 전망입니다. 미니멀리즘은 삶의 본질을 추구하는 현대 사회의 중요한 변화를 반영합니다."
    },
    {
        "Missing_Entities": "オンラインコミュニティ;SNS;新たな社会現象;価値観の変化",
        "Denser_Summary": "일본 도시 젊은층에서 물질적 소유를 최소화하고 정신적 풍요와 시간을 중시하는 '미니멀리스트' 생활 양식이 확산 중입니다. 불필요한 것을 줄여 선택지를 좁히고 본질에 집중하며, 이는 온라인 커뮤니티와 SNS를 통해 퍼져 단순하고 지속 가능한 소비를 촉진하는 새로운 사회 현상이 되었습니다. 과잉 소비 문화 반감, 경제 및 미래 불안 해소 심리가 배경이며, 클라우드 등 디지털 기술은 소유 없는 삶을 용이하게 합니다. 이러한 가치관 변화는 주택, 패션, 공공 서비스 디자인에도 영향을 미칠 것으로 예측됩니다."
    },
    {
        "Missing_Entities": "急速に拡散され;過剰な消費文化;デジタル技術の普及;予測",
        "Denser_Summary": "일본 도시 젊은층 사이에서 물질 최소화, 정신적 풍요, 시간 중시를 특징으로 하는 '미니멀리스트' 생활이 확산 중입니다. 불필요한 물건을 줄여 본질에 집중하는 이 라이프스타일은 온라인 및 SNS를 통해 빠르게 퍼지며 단순, 지속 가능한 소비를 장려하는 새 사회 현상입니다. 과잉 소비 문화 반감, 경제/미래 불안 해소가 배경이며, 클라우드 등 디지털 기술은 소유 없는 삶을 용이하게 합니다. 전문가들은 이 가치관 변화가 주택, 패션, 공공 서비스 디자인에 영향을 미칠 것이라 예측합니다."
    },
    {
        "Missing_Entities": "生活様式;最小限に抑え;本質的なこと;影響を与えるだろう",
        "Denser_Summary": "일본 도시 젊은층에서 물질적 소유를 최소화하고 정신적 풍요와 시간을 중시하는 '미니멀리스트' 생활 양식이 확산 중입니다. 불필요한 것을 줄여 선택지를 좁히고 본질에 집중하는 이 라이프스타일은 온라인 및 SNS를 통해 빠르게 퍼져 단순, 지속 가능한 소비를 장려하는 새 사회 현상입니다. 과잉 소비 문화 반감, 경제/미래 불안 해소가 배경이며, 클라우드 등 디지털 기술은 소유 없는 삶을 용이하게 합니다. 이 가치관 변화는 주택, 패션, 공공 서비스 디자인에 영향을 줄 것입니다."
    }
    ]
    ```

  * **`문장의 밀도`** 에 따라 처리 시간이 달라짐을 확인함

In [ ]:
# context_text 입력하기_6
# 토큰화가 복잡한 언어: 중국어
# 문장이 담고 있는 정보의 복잡성과 깊이 = 문장의 밀도 고려
# 중국 신문 기사 발췌문: 문화 유산의 디지털화 (文物数字化)
context_text = """
近年来，随着数字化技术的飞速发展，中国在文化遗产保护领域取得了显著进展。许多珍贵的历史文物和古建筑通过高精度三维扫描和虚拟现实（VR）技术得以永久保存，打破了时空限制，让公众能够沉浸式地体验历史。例如，故宫博物院启动了“数字故宫”项目，利用大数据和云计算技术，建立了包含百万件藏品的数字资源库。这不仅为学者提供了更为便捷的研究平台，也为游客带来了全新的参观模式。然而，数字化过程中的数据安全、版权归属以及高昂的技术成本仍然是亟待解决的难题。如何平衡保护与利用的关系，确保数字化的文化遗产能够持续为社会创造价值，是当前面临的重要课题。
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(4.5s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "디지털 기술;문화유산 보호;가상현실(VR) 기술",
        "Denser_Summary": "본문에서는 최근 디지털 기술의 비약적인 발전으로 인해 중국의 문화유산 보호 분야에서 상당한 진전이 있었음을 논하고 있다. 수많은 귀중한 역사 유물과 고대 건축물들이 고정밀 3D 스캔 및 가상현실(VR) 기술을 통해 영구적으로 보존되고 있으며, 이는 시공간의 제약을 넘어 대중이 역사에 몰입할 수 있도록 돕는다. 이러한 기술 발전은 문화유산의 보존 방식을 혁신하고 있으며, 앞으로도 이 분야의 발전이 기대된다. 디지털 기술의 발전은 문화유산 보호에 중요한 역할을 하고 있으며, 관련 기술의 적용이 확대될 것으로 예상된다. 또한, 이러한 기술들은 문화유산의 접근성을 높이는 데 기여하고 있다."
    },
    {
        "Missing_Entities": "故宫博物院;디지털故宫项目;빅데이터 및 클라우드 컴퓨팅 기술",
        "Denser_Summary": "중국의 문화유산 보호는 디지털 기술, 특히 고정밀 3D 스캔과 가상현실(VR) 기술 덕분에 크게 발전했다. 이러한 기술들은 귀중한 역사 유물과 고대 건축물을 시공간 제약 없이 영구 보존하며 대중의 몰입적 역사 체험을 가능케 한다. 대표적인 예로, 고궁박물원은 디지털 고궁 프로젝트를 통해 빅데이터 및 클라우드 컴퓨팅 기술을 활용하여 백만 건 이상의 소장품 디지털 자원 데이터베이스를 구축했다. 이는 학자들에게 편리한 연구 플랫폼을 제공하고 관광객에게는 새로운 관람 방식을 선사한다. 이러한 디지털화는 유산 접근성을 높이는 데 기여한다."
    },
    {
        "Missing_Entities": "디지털 자원 데이터베이스;연구 플랫폼;새로운 관람 방식",
        "Denser_Summary": "중국의 문화유산 보호는 고정밀 3D 스캔과 가상현실(VR) 기술의 발전으로 혁신을 이루었다. 이 기술들은 귀중한 역사 유물 및 고대 건축물을 시공간 제약 없이 영구 보존하며 대중의 몰입적 역사 체험을 지원한다. 고궁박물원의 디지털 고궁 프로젝트는 빅데이터 및 클라우드 컴퓨팅 기술을 활용해 백만 건 이상 소장품의 디지털 자원 데이터베이스를 구축했으며, 이는 학자들에게 편리한 연구 플랫폼을 제공하고 관광객에게는 새로운 관람 방식을 제시한다. 이로써 유산 접근성이 향상되고 연구 환경이 개선되었다."
    },
    {
        "Missing_Entities": "데이터 보안;저작권 귀속;높은 기술 비용",
        "Denser_Summary": "중국의 문화유산 보호는 고정밀 3D 스캔과 가상현실(VR) 기술 덕분에 시공간 제약 없이 유물을 영구 보존하고 대중의 몰입적 역사 체험을 가능케 하며 크게 발전했다. 고궁박물원의 디지털 고궁 프로젝트는 빅데이터 및 클라우드 컴퓨팅 기술로 백만 건 이상 소장품의 디지털 자원 데이터베이스를 구축, 학자에게 편리한 연구 플랫폼과 관광객에게 새로운 관람 방식을 제공했다. 그러나 이 과정에서 데이터 보안, 저작권 귀속, 높은 기술 비용 등의 문제가 여전히 중요한 과제로 남아 있다. 이러한 과제 해결은 필수적이다."
    },
    {
        "Missing_Entities": "보호와 이용의 균형;지속적인 사회적 가치 창출",
        "Denser_Summary": "중국 문화유산 보호는 고정밀 3D 스캔과 VR 기술로 유물을 영구 보존하고 몰입적 역사 체험을 제공하며 발전했다. 고궁박물원의 디지털 고궁 프로젝트는 빅데이터 및 클라우드 컴퓨팅으로 백만 건 이상 소장품 디지털 자원 데이터베이스를 구축, 학자에게 연구 플랫폼과 관광객에게 새로운 관람 방식을 제공했다. 하지만 데이터 보안, 저작권 귀속, 높은 기술 비용은 해결 과제다. 따라서 보호와 이용의 균형을 맞추고 디지털화된 문화유산이 지속적으로 사회적 가치를 창출하도록 하는 방안 모색이 현안이다."
    }
    ]
    ```

In [ ]:
# context_text 입력하기_7
# 토큰화가 복잡한 언어: 중국어
# 문장이 담고 있는 정보의 복잡성과 깊이 높여보기
# 중국 신문 기사 발췌문: 스마트시티와 도시 계획
context_text = """
随着城市化进程的加速和数字化技术的成熟，中国各城市正在积极探索“智慧城市”建设，旨在通过技术手段提升城市管理效率和服务水平。智慧城市的核心在于利用物联网(IoT)、大数据、人工智能(AI)和5G通信等前沿技术，实现城市基础设施的互联互通。例如，在交通领域，智能交通系统通过实时监控和数据分析，能够有效缓解交通拥堵，优化公共交通线路；在环境管理方面，传感器网络可以实时监测空气质量、水质等环境指标，为城市管理者提供决策依据。然而，智慧城市建设也面临诸多挑战。数据孤岛问题依然存在，不同部门之间的数据无法有效共享，影响了整体效能。此外，数据安全和公民隐私保护问题也日益突出，需要建立完善的法律法规和技术保障体系。专家们指出，未来的智慧城市不应仅仅是技术的堆砌，更应以人为本，关注居民的生活质量和幸福感，通过技术赋能打造一个更加宜居、可持续发展的城市空间。
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(7.7s)

  * json 형태로 출력됨
  
    ```json
    [
    {
        "Missing_Entities": "도시화; 디지털화; 기술; 도시 관리; 서비스 수준; 사물 인터넷(IoT); 빅데이터; 인공지능(AI); 5G 통신; 상호 연결; 교통; 교통 체증 완화; 대중교통 노선 최적화; 환경 관리; 센서 네트워크; 공기 질; 수질; 의사결정 지원; 데이터 사일로; 데이터 공유; 데이터 보안; 시민 프라이버시; 법규; 기술적 보장; 인간 중심; 삶의 질; 행복감; 기술 역량 강화; 거주 가능성; 지속 가능한 발전",
        "Denser_Summary": "이 글은 중국 도시들이 가속화되는 도시화와 성숙해지는 디지털화 속에서 기술을 활용하여 도시 관리 효율성과 서비스 수준을 향상시키기 위한 '스마트 시티' 건설을 적극적으로 모색하고 있다고 논의합니다. 스마트 시티의 핵심은 사물 인터넷(IoT), 빅데이터, 인공지능(AI), 5G 통신과 같은 첨단 기술을 활용하여 도시 기반 시설의 상호 연결을 실현하는 데 있습니다. 예를 들어, 교통 분야에서는 스마트 교통 시스템이 실시간 모니터링과 데이터 분석을 통해 교통 체증을 효과적으로 완화하고 대중교통 노선을 최적화할 수 있습니다. 환경 관리 측면에서는 센서 네트워크가 공기 질, 수질과 같은 환경 지표를 실시간으로 모니터링하여 도시 관리자들에게 의사결정 지원을 제공합니다. 그러나 스마트 시티 건설은 데이터 사일로 문제, 즉 부서 간 데이터 공유의 어려움으로 인해 전반적인 효능에 영향을 미치고 있다는 과제에 직면해 있습니다. 또한, 데이터 보안 및 시민 프라이버시 보호 문제도 점점 더 중요해지고 있으며, 이를 해결하기 위해 완벽한 법규와 기술적 보장 체계 구축이 필요합니다. 전문가들은 미래의 스마트 시티가 단순히 기술의 집약이 아니라, 인간 중심적이어야 하며 주민들의 삶의 질과 행복감을 중시하고, 기술 역량 강화를 통해 더욱 거주 가능하고 지속 가능한 도시 공간을 만들어야 한다고 강조합니다."
    },
    {
        "Missing_Entities": "스마트 교통 시스템; 실시간 모니터링; 데이터 분석",
        "Denser_Summary": "중국 도시들은 도시화와 디지털화 가속에 따라 기술 기반의 '스마트 시티' 건설을 추진하며 도시 관리 및 서비스 개선을 목표로 합니다. 스마트 시티는 IoT, 빅데이터, AI, 5G를 활용한 도시 기반 시설의 상호 연결을 핵심으로 합니다. 교통 분야에서는 스마트 교통 시스템이 실시간 모니터링과 데이터 분석을 통해 교통 체증 완화 및 대중교통 노선 최적화를 지원합니다. 환경 관리에는 센서 네트워크가 공기 질, 수질 등을 실시간 감시하여 의사결정 근거를 제공합니다. 그러나 데이터 사일로, 부서 간 데이터 공유 미흡, 데이터 보안 및 시민 프라이버시 침해는 주요 과제입니다. 이를 해결하기 위한 법규와 기술적 보장 체계 구축이 필수적입니다. 전문가들은 스마트 시티가 기술 집약을 넘어 인간 중심적이어야 하며, 삶의 질과 행복 증진, 기술 역량 강화를 통해 거주 가능하고 지속 가능한 도시를 지향해야 한다고 제언합니다."
    },
    {
        "Missing_Entities": "스마트 교통 시스템; 실시간 모니터링; 데이터 분석",
        "Denser_Summary": "중국 도시들은 도시화 및 디지털화에 따라 기술을 활용한 '스마트 시티' 건설을 통해 도시 관리 효율성과 서비스 수준 향상을 도모합니다. 스마트 시티의 핵심은 IoT, 빅데이터, AI, 5G 등 첨단 기술을 이용한 도시 기반 시설의 상호 연결입니다. 교통 분야의 스마트 교통 시스템은 실시간 모니터링과 데이터 분석으로 교통 체증 완화 및 대중교통 노선 최적화를 실현합니다. 환경 관리에서는 센서 네트워크가 공기 질, 수질 등 환경 지표를 실시간 감시하여 의사결정 지원에 기여합니다. 하지만 데이터 사일로, 데이터 공유의 어려움, 데이터 보안 및 시민 프라이버시 문제는 해결해야 할 과제이며, 이에 대한 법규와 기술적 보장 체계 마련이 시급합니다. 전문가는 미래 스마트 시티가 기술적 측면뿐 아니라 인간 중심적 가치를 추구하며, 삶의 질과 행복 증진, 기술 역량 강화를 통해 거주 가능하고 지속 가능한 도시 환경을 조성해야 한다고 강조합니다."
    },
    {
        "Missing_Entities": "데이터 사일로",
        "Denser_Summary": "도시화와 디지털화 가속으로 중국 도시들은 기술을 통해 도시 관리 효율성과 서비스 수준을 높이는 '스마트 시티' 건설을 적극 추진합니다. IoT, 빅데이터, AI, 5G 기술을 활용한 도시 인프라의 상호 연결이 핵심입니다. 스마트 교통 시스템은 실시간 모니터링과 데이터 분석으로 교통 체증을 줄이고 대중교통 노선을 최적화합니다. 센서 네트워크는 환경 관리에서 공기 질, 수질 등을 실시간 감시하여 의사결정에 활용됩니다. 그러나 데이터 사일로 현상으로 인한 부서 간 데이터 공유의 어려움과 데이터 보안, 시민 프라이버시 보호 문제는 중요한 과제이며, 관련 법규와 기술적 보장 체계 구축이 요구됩니다. 전문가들은 스마트 시티가 기술 집약을 넘어 인간 중심적이어야 하며, 삶의 질과 행복감을 높이고 기술 역량 강화를 통해 거주 가능하고 지속 가능한 도시를 만들어야 한다고 강조합니다."
    },
    {
        "Missing_Entities": "데이터 사일로",
        "Denser_Summary": "중국 도시들은 도시화 및 디지털화 가속에 따라 IoT, 빅데이터, AI, 5G 등 첨단 기술을 활용하여 도시 관리 효율성과 서비스 수준을 높이는 '스마트 시티' 건설에 힘쓰고 있습니다. 핵심은 도시 기반 시설의 상호 연결입니다. 스마트 교통 시스템은 실시간 모니터링 및 데이터 분석으로 교통 체증 완화와 대중교통 노선 최적화를 지원하며, 센서 네트워크는 환경 관리에서 공기 질, 수질 등 환경 지표를 실시간 감시하여 의사결정에 기여합니다. 하지만 데이터 사일로 문제로 인한 부서 간 데이터 공유의 비효율성, 데이터 보안 및 시민 프라이버시 보호 문제는 주요 과제이며, 이를 해결할 법규와 기술적 보장 체계 구축이 필요합니다. 전문가들은 스마트 시티가 기술 중심을 넘어 인간 중심적이어야 하며, 삶의 질과 행복 증진, 기술 역량 강화를 통해 거주 가능하고 지속 가능한 도시를 만들어야 한다고 주장합니다."
    }
    ]
    ```

<small>

---

##### 중국어 기사 1, 2의 차이점 : **문장의 밀도와 복잡성** 측면

* **문장의 밀도 측면**

  * **첫 번째 발췌문 (짧은 기사):**
    * **핵심 주제**: 문화 유산의 `디지털화`라는 하나의 큰 주제에 집중
    * **정보 구성**: 디지털 기술(`3D 스캔`, `VR`)을 통해 문화 유산이 보존되고 있다는 사실과 그 예시(故宮博物院 = 고궁박물관 의 `디지털 고궁` 프로젝트)를 간결하게 제시
    * **문장 구조**: 
      * 각 **문장이 비교적 짧고** 주제와 **예시** 를 **직접적으로 연결** 하는 구조가 많음
      * 즉, 정보의 밀도가 상대적으로 낮아 모델이 빠르게 핵심을 파악할 수 있음

  * **두 번째 발췌문 (긴 기사):**
      * **핵심 주제**: `스마트시티`라는 큰 주제 아래 여러 하위 주제를 다룸.
      * **정보 구성**: 스마트시티의 정의 → **핵심 기술(`IoT`, `빅데이터`, `AI`, `5G`)** → 교통 분야의 적용 예시 → 환경 분야의 적용 예시 → 문제점(데이터 고립, 보안) → 미래 비전(기술과 삶의 조화) 등 **여러 층위의 정보** 를 촘촘하게 담고 있음
      * **문장 구조**: 
        * **문장이 더 길고** **복잡한 접속어** (예: `然而(그러나)`, `此外(또한)`)를 사용하여 **여러 아이디어와 문제점을 연결** 
        * **한 문장 안에** **여러** 기술과 그 적용 사례, 그리고 도전 과제까지 **함께 언급** 되어 있어 정보의 밀도가 훨씬 높음

* **기타 차이점**

  * **주제 영역의 확장성**: 
    * 첫 번째 기사: `문화 유산 보존`이라는 비교적 제한된 주제에 머무름
    * 두 번째 기사: `스마트시티`라는 주제를 통해 교통, 환경, 기술, 사회적 과제 등 다양한 분야를 포괄적으로 다룸
  
  * **복잡한 개념의 사용**: 
    * 두 번째 기사에는 `数据孤岛(데이터 고립)`, `以人为本(인간 중심)`, `可持续发展(지속 가능한 발전)`과 같은 더 추상적이고 복합적인 개념들이 포함되어 있음
    * 이러한 개념들은 모델이 문맥을 더 깊이 이해해야만 정확하게 처리할 수 있음
  
  * **목적**: 
    * 두 번째 기사는 단순한 사실 전달을 넘어, **문제점을 지적** 하고 **미래 방향을 제시** 하는 **논평적인 성격** 이 강함. 
    * 이 역시 모델이 단순 요약 이상의 복합적인 작업을 수행하도록 요구함

* 결론적으로, 두 번째 중국어 기사가 더 오래 걸린 이유는 **더 넓은 범위의 주제를 다루고 있으며, 문장 구조가 복잡하고, 추상적인 개념들이 많아 정보 밀도가 훨씬 높기 때문**
* 이 때문에 모델은 더 많은 토큰을 처리하고, 각 토큰 간의 복잡한 관계를 파악하는 데 더 많은 연산 자원을 사용하게 된 것

---

<small>

* 의문? 
  * 우리가 자주 접하지 않는 언어들 (예시: 태국어, 이슬람어 등)의 토큰화는 어떨까? 
  
* 또 다른 복잡성을 가짐

  * **태국어**:
    * **띄어쓰기 부재**: 
      * 태국어는 문장 내에서 단어를 구분하기 위한 띄어쓰기 없음
      * 모든 단어가 붙어 있어서, 텍스트를 토큰화할 때 단어 경계를 정확하게 파악하는 것이 큰 과제
    * **토큰화 방식**: 
      * 따라서 태국어를 처리할 때는 텍스트를 의미 있는 단위로 나누기 위해 별도의 알고리즘이나 사전 기반의 모델이 필요
      * 이러한 과정이 영어에 비해 훨씬 복잡하고 연산 비용이 많이 들 수 있음
    * **결론**: 
      * 띄어쓰기가 없는 언어는 토큰화 과정 자체에서 추가적인 작업이 필요하므로, 일반적으로 영어보다 처리 속도가 느림

  * **아랍어**:
    * **우에서 좌로 쓰기**: 
      * 아랍어는 오른쪽에서 왼쪽으로 쓰는 독특한 표기 방식을 가짐
      * 이 자체는 토큰화에 직접적인 영향을 주기보다는, **모델의 내부 처리 파이프라인에서 방향성을 고려해야 하는 요소**
    * **형태론적 복잡성**: 
      * 아랍어는 `어근(root)`을 중심으로 `접두사`, `접미사` 등이 결합하여 단어를 형성하는 `비연속적인 어근 체계`
      * 예시: `كتب(쓰다)` 라는 어근 → `كتاب(책)`, `كاتب(작가)`, `مكتب(사무실)` 등이 파생
    * **토큰화 방식**: 
      * 이러한 **형태론적 복잡성** 때문에, 아랍어 텍스트를 토큰화할 때는 단순히 단어를 쪼개는 것을 넘어, **어근과 접사를 분리** 하고 **형태소 분석을 수행** 하는 **복잡한 과정이 필요** 
      * 이 역시 영어와 비교했을 때 연산 비용이 높을 수 있음
    * **결론**: 아랍어는 복잡한 형태론적 특징으로 인해 토큰화 과정이 더 복잡하고, 이로 인해 처리 속도에 영향을 미칠 수 있음

  * 요약: 
    * 우리에게 상대적으로 낯선 언어들 중에서도 **띄어쓰기 여부, 형태론적 복잡성, 문자 체계의 특성** 등 다양한 요인들이 토큰화 효율에 영향을 미침
    * 일반적으로, 영어처럼 **띄어쓰기가 명확** 하고 **형태론적 구조가 단순한 언어** 일수록 토큰화가 효율적이
    * **`그 외의 언어들`** = 토큰화 과정에서 추가적인 처리 비용이 발생할 가능성이 높음

In [ ]:
# context_text 입력하기_8
# 토큰화가 복잡한 언어: 태국어
# 문장이 담고 있는 정보의 복잡성과 깊이 = 문장의 밀도 고려
# 태국어 신문 기사 발췌문: 지속 가능한 관광 (การท่องเที่ยวเชิงอนุรักษ์)
context_text = """
ภาวะโลกร้อนและปัญหาสิ่งแวดล้อมที่ทวีความรุนแรงขึ้น ได้ส่งผลกระทบโดยตรงต่ออุตสาหกรรมการท่องเที่ยวทั่วโลก ทำให้แนวคิดการท่องเที่ยวเชิงอนุรักษ์กลายเป็นกระแสหลัก ผู้ประกอบการและนักท่องเที่ยวหันมาให้ความสำคัญกับการลดการปล่อยคาร์บอนและผลกระทบต่อธรรมชาติมากขึ้น หลายภูมิภาคในประเทศไทยกำลังปรับเปลี่ยนกลยุทธ์จาก "การท่องเที่ยวเชิงมวลชน" ไปสู่ "การท่องเที่ยวเชิงคุณภาพ" โดยเน้นการสร้างประสบการณ์ที่ลึกซึ้งและมีความรับผิดชอบต่อสิ่งแวดล้อมและวัฒนธรรมท้องถิ่น แทนที่จะเน้นปริมาณนักท่องเที่ยว การเปลี่ยนแปลงนี้รวมถึงการส่งเสริมโฮมสเตย์ชุมชน, การใช้วัสดุที่เป็นมิตรต่อสิ่งแวดล้อม และการจัดการขยะอย่างมีประสิทธิภาพ ผู้เชี่ยวชาญชี้ว่าแนวทางนี้ไม่เพียงแต่ช่วยรักษาระบบนิเวศ แต่ยังช่วยกระจายรายได้สู่ชุมชนท้องถิ่นอย่างยั่งยืน และเสริมสร้างภาพลักษณ์ที่ดีให้กับประเทศในระยะยาว นอกจากนี้ยังมีการพัฒนาแพลตฟอร์มออนไลน์ที่ช่วยให้นักท่องเที่ยวสามารถค้นหาและจองกิจกรรมท่องเที่ยวเชิงอนุรักษ์ได้ง่ายขึ้น ซึ่งช่วยกระตุ้นให้ตลาดเติบโตอย่างต่อเนื่อง และทำให้ผู้บริโภคมีทางเลือกที่หลากหลายมากขึ้นในการตัดสินใจเดินทาง
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 
  * 입력된 내용 
    
    ```<plaintext>
    
    ภาวะโลกร้อนและปัญหาสิ่งแวดล้อมที่ทวีความรุนแรงขึ้น ได้ส่งผลกระทบโดยตรงต่ออุตสาหกรรมการท่องเที่ยวทั่วโลก ทำให้แนวคิดการท่องเที่ยวเชิงอนุรักษ์กลายเป็นกระแสหลัก ผู้ประกอบการและนักท่องเที่ยวหันมาให้ความสำคัญกับการลดการปล่อยคาร์บอนและผลกระทบต่อธรรมชาติมากขึ้น หลายภูมิภาคในประเทศไทยกำลังปรับเปลี่ยนกลยุทธ์จาก "การท่องเที่ยวเชิงมวลชน" ไปสู่ "การท่องเที่ยวเชิงคุณภาพ" โดยเน้นการสร้างประสบการณ์ที่ลึกซึ้งและมีความรับผิดชอบต่อสิ่งแวดล้อมและวัฒนธรรมท้องถิ่น แทนที่จะเน้นปริมาณนักท่องเที่ยว การเปลี่ยนแปลงนี้รวมถึงการส่งเสริมโฮมสเตย์ชุมชน, การใช้วัสดุที่เป็นมิตรต่อสิ่งแวดล้อม และการจัดการขยะอย่างมีประสิทธิภาพ ผู้เชี่ยวชาญชี้ว่าแนวทางนี้ไม่เพียงแต่ช่วยรักษาระบบนิเวศ แต่ยังช่วยกระจายรายได้สู่ชุมชนท้องถิ่นอย่างยั่งยืน และเสริมสร้างภาพลักษณ์ที่ดีให้กับประเทศในระยะยาว
    
    ```

  * 한국어 해석 
    
    ```<plaintext>
    
    지구 온난화와 악화되는 환경 문제는 세계 관광 산업에 직접적인 영향을 미치며 생태 관광을 주류로 끌어올렸습니다. 관광업체와 관광객 모두 탄소 배출량 감축과 자연에 미치는 영향 감소를 점점 더 중요하게 여기고 있습니다. 태국의 많은 지역이 "대량 관광"에서 "양질 관광"으로 전략을 전환하고 있으며, 관광객 수보다는 몰입형 경험을 창출하고 환경과 지역 문화에 대한 책임을 다하는 데 중점을 두고 있습니다. 이러한 변화에는 지역 사회 기반 홈스테이 활성화, 친환경 자재 사용, 효율적인 폐기물 관리 등이 포함됩니다. 전문가들은 이러한 접근 방식이 생태계를 보존할 뿐만 아니라 지역 사회에 지속 가능한 소득을 분배하고 장기적으로 국가의 이미지를 향상시킨다고 지적합니다.
    
    ```

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)

  * 영어(5.7s)
  
  * 태국어(6.0s)
  
    ```json
    [
    {
        "Missing_Entities": "ภาวะโลกร้อน; ผลกระทบต่ออุตสาหกรรมการท่องเที่ยว; การท่องเที่ยวเชิงอนุรักษ์",
        "Denser_Summary": "본 문서는 최근 심화되고 있는 지구 온난화와 환경 문제들이 전 세계 관광 산업에 직접적인 영향을 미치고 있으며, 이로 인해 관광 방식의 근본적인 변화가 일어나고 있음을 상세히 설명하고 있습니다. 특히, 과거 대규모 관광객 유치에 집중했던 방식에서 벗어나, 점차 환경 및 문화적 책임감을 강조하는 새로운 관광 패러다임으로 전환되고 있다는 점을 부각하고 있습니다. 이러한 변화는 관광객들에게 보다 깊이 있고 지속 가능한 경험을 제공하는 것을 목표로 하며, 이는 지역 사회의 발전과 환경 보존이라는 두 가지 중요한 가치를 동시에 추구하는 방향으로 나아가고 있음을 시사합니다. 따라서 본 문서는 이러한 추세가 관광 산업 전반에 걸쳐 어떻게 나타나고 있는지, 그리고 이것이 가져올 긍정적인 효과는 무엇인지에 대한 포괄적인 개요를 제공합니다. 결국, 이러한 새로운 관광 흐름은 지속 가능한 발전을 위한 필수적인 요소로 자리매김하고 있다고 할 수 있습니다."
    },
    {
        "Missing_Entities": "การลดการปล่อยคาร์บอน; ผลกระทบต่อธรรมชาติ; การท่องเที่ยวเชิงคุณภาพ",
        "Denser_Summary": "본 문서는 심화되는 지구 온난화와 환경 문제가 전 세계 관광 산업에 직접적인 영향을 미치면서, '관광의 질'을 중시하는 '관광의 질'로의 전환이 가속화되고 있음을 강조합니다. 이는 대규모 관광객 유치에서 벗어나, 탄소 배출 감소 및 자연에 미치는 영향을 최소화하는 책임감 있는 관광을 추구하는 경향을 반영합니다. 이러한 변화는 관광객들에게 깊이 있고 책임감 있는 경험을 제공하는 데 초점을 맞추며, 이는 전 세계적으로 관광 산업의 새로운 패러다임으로 부상하고 있음을 시사합니다. 결과적으로, 이러한 전환은 환경 보호와 지속 가능한 발전을 동시에 달성하려는 노력의 일환으로 볼 수 있습니다."
    },
    {
        "Missing_Entities": "โฮมสเตย์ชุมชน; วัสดุที่เป็นมิตรต่อสิ่งแวดล้อม; การจัดการขยะอย่างมีประสิทธิภาพ",
        "Denser_Summary": "본 문서는 심화되는 지구 온난화와 환경 문제가 전 세계 관광 산업에 직접적인 영향을 미치면서, '관광의 질'을 중시하는 '관광의 질'로의 전환이 가속화되고 있음을 강조합니다. 이는 대규모 관광객 유치에서 벗어나, 탄소 배출 감소 및 자연에 미치는 영향을 최소화하는 책임감 있는 관광을 추구하는 경향을 반영합니다. 이러한 변화는 관광객들에게 깊이 있고 책임감 있는 경험을 제공하는 데 초점을 맞추며, 지역 사회의 발전과 환경 보존이라는 두 가지 중요한 가치를 동시에 추구하는 방향으로 나아가고 있음을 시사합니다. 특히, 지역 경제 활성화를 위한 홈스테이 장려, 친환경 소재 사용, 효율적인 폐기물 관리 등 구체적인 실천 방안들이 논의되고 있음을 알 수 있습니다."
    },
    {
        "Missing_Entities": "กระจายรายได้สู่ชุมชนท้องถิ่นอย่างยั่งยืน; เสริมสร้างภาพลักษณ์ที่ดีให้กับประเทศในระยะยาว; แพลตฟอร์มออนไลน์",
        "Denser_Summary": "본 문서는 심화되는 지구 온난화와 환경 문제가 전 세계 관광 산업에 직접적인 영향을 미치면서, '관광의 질'을 중시하는 '관광의 질'로의 전환이 가속화되고 있음을 강조합니다. 이는 대규모 관광객 유치에서 벗어나, 탄소 배출 감소 및 자연에 미치는 영향을 최소화하는 책임감 있는 관광을 추구하는 경향을 반영합니다. 이러한 변화는 관광객들에게 깊이 있고 책임감 있는 경험을 제공하는 데 초점을 맞추며, 지역 사회의 발전과 환경 보존이라는 두 가지 중요한 가치를 동시에 추구하는 방향으로 나아가고 있음을 시사합니다. 특히, 홈스테이 장려, 친환경 소재 사용, 효율적인 폐기물 관리 등은 지속 가능한 지역 소득 분배와 국가 이미지 제고에 기여할 수 있습니다. 더불어, 온라인 플랫폼 개발은 이러한 친환경 관광 활동에 대한 접근성을 높여 시장 성장을 촉진합니다."
    },
    {
        "Missing_Entities": "การค้นหาและจองกิจกรรมท่องเที่ยวเชิงอนุรักษ์; กระตุ้นให้ตลาดเติบโตอย่างต่อเนื่อง; ผู้บริโภคมีทางเลือกที่หลากหลายมากขึ้น",
        "Denser_Summary": "본 문서는 심화되는 지구 온난화와 환경 문제가 전 세계 관광 산업에 직접적인 영향을 미치면서, '관광의 질'을 중시하는 '관광의 질'로의 전환이 가속화되고 있음을 강조합니다. 이는 대규모 관광객 유치에서 벗어나, 탄소 배출 감소 및 자연에 미치는 영향을 최소화하는 책임감 있는 관광을 추구하는 경향을 반영합니다. 이러한 변화는 관광객들에게 깊이 있고 책임감 있는 경험을 제공하는 데 초점을 맞추며, 지역 사회의 발전과 환경 보존이라는 두 가지 중요한 가치를 동시에 추구하는 방향으로 나아가고 있음을 시사합니다. 홈스테이 장려, 친환경 소재 사용, 효율적인 폐기물 관리는 지속 가능한 지역 소득 분배와 국가 이미지 제고에 기여하며, 온라인 플랫폼은 친환경 관광 활동 검색 및 예약 편의성을 높여 시장 성장을 촉진하고 소비자 선택권을 확대합니다."
    }
    ]
    ```

In [ ]:
# context_text 입력하기_9
# 토큰화가 복잡한 언어: 이슬람어
# 문장이 담고 있는 정보의 복잡성과 깊이 = 문장의 밀도 고려
# 아랍어 신문 기사 발췌문: 공중 보건 및 디지털 혁신 (الصحة العامة والتحول الرقمي)
context_text = """
تشهد الأنظمة الصحية في منطقة الشرق الأوسط تحولاً كبيراً، حيث تتجه العديد من الدول نحو تبني الحلول الرقمية لتحسين جودة الرعاية الصحية وزيادة كفاءة الخدمات. من أبرز هذه الحلول استخدام السجلات الصحية الإلكترونية، التي تسمح للأطباء بالوصول الفوري إلى تاريخ المريض الطبي، مما يقلل من الأخطاء الطبية ويسرع عملية التشخيص. كما تلعب تطبيقات الصحة المتنقلة دوراً حيوياً في تمكين الأفراد من مراقبة حالتهم الصحية وتلقي استشارات طبية عن بُعد، خاصة في المناطق النائية. هذا التحول الرقمي يساهم أيضاً في جمع كميات هائلة من البيانات الصحية التي يمكن تحليلها للتنبؤ بالأوبئة وتخطيط الموارد الصحية بشكل أكثر فعالية. ومع ذلك، لا تزال هناك تحديات كبيرة تتعلق بضمان أمن البيانات وخصوصية المرضى، وتوفير البنية التحتية اللازمة للتحول الرقمي في جميع المؤسسات الصحية، مما يتطلب استثمارات ضخمة وجهوداً مشتركة من القطاعين العام والخاص. على صعيد آخر، تم إطلاق العديد من المبادرات الحكومية لتدريب الكوادر الطبية على استخدام هذه التقنيات الجديدة، مما يضمن أن يكون العاملون في القطاع الصحي مستعدين للمستقبل الرقمي. بالإضافة إلى ذلك، تعمل شركات التكنولوجيا على تطوير حلول قائمة على الذكاء الاصطناعي لتحليل الصور الطبية وتقديم توصيات علاجية أولية، مما يعزز قدرة الأطباء على اتخاذ قرارات دقيقة وسريعة.
"""

# {ARTICLE}
result = chain_density.invoke({"ARTICLE": context_text})
print(result)

<small>

* 
  * 입력 내용
  ```<plaintext>
  تشهد الأنظمة الصحية في منطقة الشرق الأوسط تحولاً كبيراً، حيث تتجه العديد من الدول نحو تبني الحلول الرقمية لتحسين جودة الرعاية الصحية وزيادة كفاءة الخدمات. من أبرز هذه الحلول استخدام السجلات الصحية الإلكترونية، التي تسمح للأطباء بالوصول الفوري إلى تاريخ المريض الطبي، مما يقلل من الأخطاء الطبية ويسرع عملية التشخيص. كما تلعب تطبيقات الصحة المتنقلة دوراً حيوياً في تمكين الأفراد من مراقبة حالتهم الصحية وتلقي استشارات طبية عن بُعد، خاصة في المناطق النائية. هذا التحول الرقمي يساهم أيضاً في جمع كميات هائلة من البيانات الصحية التي يمكن تحليلها للتنبؤ بالأوبئة وتخطيط الموارد الصحية بشكل أكثر فعالية. ومع ذلك، لا تزال هناك تحديات كبيرة تتعلق بضمان أمن البيانات وخصوصية المرضى، وتوفير البنية التحتية اللازمة للتحول الرقمي في جميع المؤسسات الصحية، مما يتطلب استثمارات ضخمة وجهوداً مشتركة من القطاعين العام والخاص.
  ```

  * 한국어 번역
  ```<plaintext>
  중동의 의료 시스템은 큰 변화를 겪고 있으며, 많은 국가들이 의료 서비스의 질을 개선하고 서비스 효율성을 높이기 위해 디지털 솔루션을 도입하고 있습니다. 이러한 솔루션 중 가장 눈에 띄는 것은 전자 건강 기록(EHR) 활용입니다. EHR은 의사가 환자의 병력에 즉시 접근하여 의료 오류를 줄이고 진단 과정을 가속화할 수 있도록 지원합니다. 모바일 건강 애플리케이션은 특히 외딴 지역에서 개인이 자신의 건강을 모니터링하고 원격 진료를 받을 수 있도록 하는 데 중요한 역할을 합니다. 이러한 디지털 혁신은 방대한 양의 건강 데이터를 수집하여 분석하고 전염병을 예측하고 의료 자원을 더욱 효과적으로 계획하는 데 기여합니다. 그러나 데이터 보안 및 환자 개인 정보 보호 보장, 그리고 모든 의료 기관에 디지털 혁신에 필요한 인프라 구축과 관련하여 여전히 상당한 과제가 남아 있습니다. 이를 위해서는 공공 부문과 민간 부문의 막대한 투자와 공동 노력이 필요합니다.
  ```

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)
  
  * 태국어(6.0s)
  
  * 이슬람어 (5.2s)
  
    ```json
    [
    {
        "Missing_Entities": "디지털 전환;의료 서비스 품질;전자 건강 기록",
        "Denser_Summary": "이 글은 중동 지역의 보건 시스템이 상당한 변화를 겪고 있음을 논하며, 많은 국가들이 의료 서비스 품질을 향상시키고 효율성을 증대시키기 위해 디지털 솔루션을 채택하는 방향으로 나아가고 있다고 설명합니다. 이러한 추세의 주요 특징 중 하나는 전자 건강 기록의 사용 증가인데, 이는 의료진이 환자의 의료 기록에 즉시 접근할 수 있도록 하여 의료 오류를 줄이고 진단 과정을 가속화하는 데 기여합니다. 또한, 모바일 건강 애플리케이션은 개인이 자신의 건강 상태를 모니터링하고 원격으로 의료 상담을 받을 수 있도록 하는 데 중요한 역할을 합니다. 이 디지털 전환은 또한 전염병을 예측하고 의료 자원을 보다 효과적으로 계획하기 위해 분석될 수 있는 방대한 양의 건강 데이터를 수집하는 데 기여합니다. 그러나 데이터 보안 및 환자 개인 정보 보호를 보장하는 것과 같은 상당한 과제가 여전히 남아 있으며, 모든 보건 기관에서 디지털 전환에 필요한 인프라를 제공하는 데에는 상당한 투자가 필요합니다. 마지막으로, 정부는 이러한 새로운 기술을 사용할 의료 인력을 교육하기 위한 많은 이니셔티브를 시작하여 보건 분야 종사자들이 디지털 미래를 준비하도록 보장하고 있습니다."
    },
    {
        "Missing_Entities": "원격 의료;데이터 분석;건강 자원 계획",
        "Denser_Summary": "중동 보건 시스템은 디지털 전환을 통해 의료 서비스 품질 향상과 효율 증대를 추구하고 있으며, 전자 건강 기록은 의료진의 즉각적인 환자 기록 접근을 가능케 하여 오류 감소 및 진단 가속화에 기여합니다. 모바일 건강 앱은 개인의 건강 모니터링과 원격 의료 상담을 지원하며, 특히 외딴 지역 주민들에게 유용합니다. 이러한 디지털화는 전염병 예측 및 건강 자원 계획에 활용될 방대한 건강 데이터 분석을 가능하게 합니다. 하지만 데이터 보안, 환자 개인 정보 보호, 그리고 모든 보건 기관에 필요한 인프라 구축은 여전히 중요한 과제로 남아 있으며, 이를 위해 공공 및 민간 부문의 대규모 투자와 협력이 요구됩니다. 또한, 정부는 의료 인력의 디지털 기술 교육을 위한 이니셔티브를 시행하여 미래 대비를 강화하고 있습니다."
    },
    {
        "Missing_Entities": "의료 오류 감소;진단 과정 가속화;외딴 지역",
        "Denser_Summary": "중동 보건 시스템은 디지털 전환을 통해 의료 서비스 품질을 높이고 효율성을 극대화하고 있습니다. 전자 건강 기록은 의료진에게 환자 기록에 대한 즉각적인 접근을 제공하여 의료 오류를 줄이고 진단 과정을 가속화합니다. 모바일 건강 앱은 개인의 건강 관리를 지원하고, 특히 외딴 지역 주민들에게 원격 의료 상담을 제공하는 데 중요한 역할을 합니다. 이 디지털화는 전염병 예측 및 건강 자원 계획을 위한 대규모 건강 데이터 분석을 촉진합니다. 그러나 데이터 보안, 환자 개인 정보 보호, 그리고 모든 보건 기관에 필요한 인프라 구축은 상당한 투자와 협력이 필요한 과제입니다. 정부는 의료 인력의 디지털 기술 교육을 위한 이니셔티브를 통해 미래를 준비하고 있습니다."
    },
    {
        "Missing_Entities": "개인 정보 보호;인프라 구축;의료 인력 교육",
        "Denser_Summary": "중동 보건 시스템은 디지털 전환으로 의료 품질 향상과 효율 증대를 꾀하며, 전자 건강 기록은 오류 감소 및 진단 가속화를 지원합니다. 모바일 건강 앱은 개인 건강 관리와 외딴 지역 원격 의료 상담을 강화합니다. 이 디지털화는 전염병 예측 및 건강 자원 계획을 위한 데이터 분석을 가능하게 합니다. 데이터 보안, 환자 개인 정보 보호, 그리고 모든 보건 기관의 인프라 구축은 대규모 투자와 협력이 필요한 과제이며, 정부는 의료 인력 교육 이니셔티브를 통해 미래를 준비하고 있습니다."
    },
    {
        "Missing_Entities": "AI 기반 솔루션;의학 영상 분석;치료 권고",
        "Denser_Summary": "중동 보건 시스템은 디지털 전환을 통해 의료 품질 향상과 효율 증대를 추구합니다. 전자 건강 기록은 오류 감소 및 진단 가속화를 지원하고, 모바일 건강 앱은 개인 건강 관리와 외딴 지역 원격 의료 상담을 강화합니다. 디지털화는 전염병 예측 및 건강 자원 계획을 위한 데이터 분석을 가능하게 합니다. 데이터 보안, 개인 정보 보호, 인프라 구축은 투자와 협력이 필요한 과제이며, 정부는 의료 인력 교육을 통해 미래를 준비합니다. 또한, AI 기반 솔루션은 의학 영상 분석 및 초기 치료 권고를 제공하여 의사의 의사 결정 능력을 향상시킵니다."
    }
    ]
    ```

  * `Missing_Entities`에 대한 처리가 다른 이유?
      * 아랍어: 기사 내용을 한국어로 이해하고 요약문, 추출된 핵심 엔티티도 모두 한국어로 번역해 출력
      * 태국어: 핵심 엔티티를 태국어 원문으로 출력

    * 가능성_1 =프롬프트 템플릿에 명시된 지시사항 때문: `Use only KOREAN language to reply.` 

    * 가능성_2 모델의 언어별 내부 처리 방식이 다름
      * **언어별 토큰화** 와 **내부 처리의 차이**: 
        * 아랍어: 복잡한 형태론적 특징을 가지고 있어, 모델이 내부적으로 **`형태소 단위`** 로 분해하고 의미를 파악하는 과정에서 **`한국어와 비슷`** 한 **`의미 단위`** 로 묶어 번역하기 용이했을 수 있음
        * 태국어: **띄어쓰기 없는 특성** 때문에 모델이 단어 경계를 끊는 것보다는 **원문을 그대로 가져오는 것** 이 **더 효율적이라고 판단했을 가능성** 이 있음

    * 가능성_3 특정 언어에 대한 프롬프트 지시사항의 해석 방식이 미묘하게 다를 수 있음을 시사
      * **프롬프트 지시의 미묘한 해석**: 
        * `Use only KOREAN language to reply.`라는 지시가 `최종 결과물(요약)`에만 강력하게 적용되고, `중간 과정에서 추출된 엔티티`에는 덜 엄격하게 적용되었을 수도 있음
        * 특히 모델이 `아랍어`보다 `태국어-한국어` 간의 직접적인 엔티티 번역에 대해 `덜 최적화`되어 있거나, 원문을 그대로 가져오는 것을 더 `충실한(faithful)` 응답으로 판단했을 가능성도 있음

<small>

* *참고: [Analysis of the language-specific processing speed and operating principles of LLM](../docs/insights/Analysis%20of%20the%20language-specific%20processing%20speed%20and%20operating%20principles%20of%20LLM.md)*

---

#### **`7)Chain of Density (한국어) - 2 (텍스트 버전)`**

* 역할
  * 위와 같은 `Chain of Density` 요약이지만, `3단계`만 반복하고 JSON 대신 `텍스트 형식`으로 결과를 줌
  * 한국어로 단계별 Missing Entities(빠진 엔티티)와 요약문을 보여줌

In [18]:
prompt_title = "chain-of-density-map-korean"

prompt_template = """Article: {ARTICLE}
You will generate increasingly concise, entity-dense summaries of the above article. 

Repeat the following 2 steps 3 times. 

Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. 
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. 

A missing entity is:
- relevant to the main story, 
- specific yet concise (100 words or fewer), 
- novel (not in the previous summary), 
- faithful (present in the article), 
- anywhere (can be located anywhere in the article).

Guidelines:

- The first summary should be long (8-10 sentences, ~200 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~200 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article. 
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities. 

Remember, use the exact same number of words for each summary.
Answer "Missing Entities" and "Denser_Summary" as in TEXT format.
Use only KOREAN language to reply."""

prompt_density_text = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
# 허브에 올리기 
result_density_text = client.push_prompt("prompt_template_density_text", object=prompt_density_text)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_density_text)
print('\n', "-" * 50)

# 허브에서 가져오기 
prompt_density_text_from_hub = client.pull_prompt("prompt_template_density_text")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")

# 최근 커밋 해시 조회하기
prompt_id = "prompt_template_density_text"
prompt_info_density_text = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_density_text)

prompt_info_density_text = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_density_text.__dict__.items():
    print(f"{key}: {value}")

<small>

* 셀 출력

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_density_text/0b372530?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.
    프롬프트 정보: repo_handle='prompt_template_density_text' description='' readme='' id='f7fbd96d-ebf5-4ac5-95ba-358dbfa85b4a' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 14, 3, 17, 16, 101122) updated_at=datetime.datetime(2025, 8, 14, 3, 17, 17, 810829) is_public=False is_archived=False tags=['ChatPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_density_text' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='0b3725303f31b1226388e38007fb1a0cb74a6c6cce7f54e9112e763fda4ec5df' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_density_text
    description: 
    readme: 
    id: f7fbd96d-ebf5-4ac5-95ba-358dbfa85b4a
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-14 03:17:16.101122
    updated_at: 2025-08-14 03:17:17.810829
    is_public: False
    is_archived: False
    tags: ['ChatPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_density_text
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: 0b3725303f31b1226388e38007fb1a0cb74a6c6cce7f54e9112e763fda4ec5df
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None
    ```

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc               # 위에서 정의함

# chain_map 구성
chain_density_text = prompt_density_text | gemini_lc | StrOutputParser()

# context_text 입력하기
# 한국어 기사 예시: 문화유산
context_text = """
신라 불교 예술의 정수로 평가받는 석굴암은 통일신라 시대 경덕왕 시절 재상 김대성에 의해 창건되었다. 석굴암은 불국사와 함께 신라의 이상적인 불국토를 현세에 구현하려는 목적으로 지어졌으며, 토함산 중턱에 인공으로 만들어진 석굴은 주실 본존불을 중심으로 다양한 불상과 보살상들이 조화롭게 배치되어 있다. 특히, 주실 내부에 자리한 본존불은 신라의 이상적인 불국토를 구현한 것으로, 완벽한 비례와 사실적인 조각 기술을 보여준다. 석굴의 주벽에는 십일면 관음보살상과 십대 제자, 사천왕상이 정교하게 새겨져 있어 그 예술적 가치를 더한다. 석굴암은 단순히 종교적인 공간을 넘어, 신라의 뛰어난 건축 기술과 수학적 원리가 결합된 과학적인 구조물로도 인정받는다. 석굴의 습도와 온도를 일정하게 유지하는 배수 시설과 환기 체계는 1,000년이 넘는 시간 동안 내부를 보존하는 데 핵심적인 역할을 해왔다. 이러한 복합적인 기술과 예술성이 어우러진 석굴암은 1995년 유네스코 세계유산으로 등재되며 그 가치를 세계적으로 인정받았다.
"""

# {ARTICLE}
result = chain_density_text.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(4.2s)

  * text 형태로 출력됨
  
    ```<plaintext>
    Missing Entities:
    불국사;통일신라 시대;김대성

    Denser_Summary:
    이 글은 신라 불교 예술의 정수로 평가받는 석굴암에 대해 다루고 있습니다. 석굴암은 통일신라 시대의 중요한 문화유산으로, 경덕왕 시절 재상 김대성에 의해 창건되었습니다. 석굴암은 불국사와 함께 신라의 이상적인 불국토를 현세에 구현하려는 목적으로 지어졌습니다. 토함산 중턱에 위치한 이 인공 석굴은 주실의 본존불을 중심으로 다양한 불상과 보살상이 조화롭게 배치되어 있습니다. 특히 본존불은 완벽한 비례와 사실적인 조각 기술을 보여주며 신라의 이상을 구현했습니다. 또한, 석굴의 주벽에는 십일면 관음보살상, 십대 제자, 사천왕상 등이 정교하게 새겨져 있어 예술적 가치를 높입니다. 석굴암은 단순한 종교 공간을 넘어, 신라의 뛰어난 건축 기술과 수학적 원리가 결합된 과학적인 구조물로도 인정받습니다. 내부의 습도와 온도를 일정하게 유지하는 배수 시설과 환기 체계는 천 년 이상 보존될 수 있었던 핵심적인 역할을 했습니다. 이러한 복합적인 기술과 예술성이 어우러진 석굴암은 1995년 유네스코 세계유산으로 등재되며 그 가치를 세계적으로 인정받았습니다.

    Missing Entities:
    십일면 관음보살상;십대 제자;사천왕상

    Denser_Summary:
    신라 불교 예술의 정수로 평가받는 석굴암은 통일신라 시대 경덕왕 시절 재상 김대성에 의해 창건되었습니다. 불국사와 함께 신라의 이상적인 불국토를 현세에 구현하려는 목적으로 지어진 석굴암은 토함산 중턱에 인공으로 조성되었습니다. 주실 본존불을 중심으로 다양한 불상과 보살상이 조화롭게 배치되어 있으며, 특히 본존불은 완벽한 비례와 사실적인 조각 기술로 신라의 이상을 구현했습니다. 석굴의 주벽에는 십일면 관음보살상, 십대 제자, 사천왕상 등이 정교하게 새겨져 예술적 가치를 더합니다. 석굴암은 단순한 종교 공간을 넘어, 신라의 뛰어난 건축 기술과 수학적 원리가 결합된 과학적인 구조물로도 인정받습니다. 천 년 이상 내부를 보존해온 습도 및 온도 조절 배수 시설과 환기 체계는 그 과학성을 보여줍니다. 복합적인 기술과 예술성이 어우러진 석굴암은 1995년 유네스코 세계유산으로 등재되었습니다.

    Missing Entities:
    배수 시설;환기 체계;1995년

    Denser_Summary:
    신라 불교 예술의 정수 석굴암은 통일신라 경덕왕 시기 김대성이 창건했습니다. 불국사와 함께 신라 이상 불국토 구현 목적으로 토함산 중턱에 조성된 석굴로, 주실 본존불 중심의 불상, 보살상 배치가 특징입니다. 본존불은 완벽 비례와 사실적 조각으로 신라 이상을 구현했으며, 주벽의 십일면 관음보살상, 십대 제자, 사천왕상은 예술성을 더합니다. 석굴암은 신라 건축 기술과 수학 원리가 결합된 과학적 구조물로, 습도·온도 유지 배수 시설과 환기 체계는 1,000년 이상 보존의 핵심입니다. 복합 기술과 예술성의 조화로 1995년 유네스코 세계유산에 등재되었습니다.
    ```

In [ ]:
# chain_map 구성
chain_density_text = prompt_density_text | gemini_lc | StrOutputParser()

# context_text 입력하기
# 영어 기사 예시: 과학
context_text = """
A recent study published in the journal "Nature Neuroscience" has shed new light on the mechanisms of memory formation. Researchers at the University of California discovered that a specific protein, identified as ARC, plays a crucial role in strengthening synaptic connections between neurons. These connections, known as synapses, are the fundamental basis for learning and memory. The study, which involved observing neural activity in mice, found that ARC helps to physically reshape dendrites—the branch-like structures of neurons that receive signals—in response to new experiences. This structural change makes the neurons more responsive and efficient, thereby solidifying the memory. The findings highlight a previously unknown pathway for neuroplasticity, the brain's ability to reorganize itself by forming new neural connections. This discovery could have significant implications for understanding memory disorders such as Alzheimer's and developing new therapeutic strategies aimed at enhancing brain function and combating cognitive decline. Future research will focus on whether these mechanisms can be manipulated to improve memory recall.
"""

# {ARTICLE}
result = chain_density_text.invoke({"ARTICLE": context_text})
print(result)

<small>

* 셀 출력 (temperature=0.7, max_tokens=4,096)(3.8s)

  * text 형태로 출력됨
  
    ```<plaintext>
    Missing Entities:
    ARC; University of California; Nature Neuroscience

    Denser_Summary:
    이 글은 기억 형성에 대한 새로운 통찰력을 제공하는 최근 연구에 대해 논의합니다. 캘리포니아 대학교의 연구원들은 "Nature Neuroscience" 저널에 발표된 연구를 통해 특정 단백질인 ARC가 뉴런 간의 시냅스 연결을 강화하는 데 중요한 역할을 한다는 사실을 발견했습니다. 이러한 시냅스 연결은 학습과 기억의 근본적인 기반입니다. 이 연구는 쥐의 신경 활동을 관찰했으며, ARC가 새로운 경험에 반응하여 뉴런의 신호를 수신하는 가지 모양 구조인 수상돌기를 물리적으로 재구성하는 데 도움을 준다는 것을 발견했습니다. 이러한 구조적 변화는 뉴런을 더 민감하고 효율적으로 만들어 기억을 공고히 합니다. 이러한 발견은 뇌가 새로운 신경 연결을 형성함으로써 스스로를 재구성하는 능력인 신경가소성에 대한 이전에 알려지지 않은 경로를 강조합니다. 이 발견은 알츠하이머와 같은 기억 장애를 이해하고 뇌 기능을 향상시키고 인지 저하에 대처하기 위한 새로운 치료 전략을 개발하는 데 중요한 영향을 미칠 수 있습니다. 미래 연구는 이러한 메커니즘이 기억 인출을 개선하기 위해 조작될 수 있는지 여부에 초점을 맞출 것입니다.

    Missing Entities:
    시냅스; 수상돌기; 신경가소성

    Denser_Summary:
    이 글은 기억 형성에 대한 중요한 발견을 보고합니다. 캘리포니아 대학교 연구원들은 "Nature Neuroscience"에 ARC라는 특정 단백질이 뉴런 간 시냅스 연결 강화에 필수적임을 확인했습니다. 이 시냅스는 학습과 기억의 기초입니다. 쥐 연구에서 ARC는 새로운 경험에 따라 뉴런의 수상돌기를 재구성하여 반응성과 효율성을 높이고 기억을 공고히 하는 것으로 밝혀졌습니다. 이 발견은 신경가소성, 즉 뇌의 재구성 능력에 대한 새로운 경로를 보여줍니다. 이는 알츠하이머와 같은 기억 장애 이해 및 뇌 기능 향상 치료 전략 개발에 중요합니다. 향후 연구는 기억 인출 향상을 위한 ARC 메커니즘 조작 가능성에 초점을 맞출 것입니다.

    Missing Entities:
    알츠하이머; 기억 인출; 뇌 기능

    Denser_Summary:
    기억 형성에 대한 캘리포니아 대학교의 "Nature Neuroscience" 연구는 ARC 단백질의 역할을 밝혀냈습니다. ARC는 뉴런 간 시냅스 연결 강화에 핵심적이며, 이는 학습과 기억의 기초입니다. 쥐 실험에서 ARC는 수상돌기 재구성을 통해 기억 공고화에 기여하며 신경가소성을 촉진합니다. 이 발견은 알츠하이머와 같은 기억 장애 이해 및 뇌 기능 향상 치료 전략 개발에 중요합니다. 향후 연구는 기억 인출 향상을 위한 ARC 메커니즘 조작 가능성을 탐구할 것입니다.
    ```

---

#### **`8) RAG 문서 프롬프트 (질문-답변용)`**

* 역할
  * 주어진 문맥(`context`)과 질문(`question`)을 받아서, **문맥 안에서 질문에 대한 답변을 찾아서 알려줌**
  * 문맥에 없으면 `정보를 찾을 수 없다`고 답변

* 특징
  * 기술용어는 번역하지 않고 그대로 사용
  * 답변은 간단명료하게, **절차적 사고(`think step-by-step`)** 반영

In [22]:
prompt_title = "rag-prompt-korean"

system = """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. Don't narrate the answer, just answer the question. Let's think step-by-step."""

human = """#Question: 
{question} 

#Context: 
{context} 

#Answer:"""

from langchain.prompts import ChatPromptTemplate

prompt_rag = ChatPromptTemplate.from_messages([
    ("system", system),
    ("human", human)
])

In [ ]:
# 허브에 올리기 
result_rag = client.push_prompt("prompt_template_rag", object=prompt_rag)
print('\n', "-" * 50, '\n')
print("✅ 프롬프트 업로드 결과:", result_rag)
print('\n', "-" * 50)

# 허브에서 가져오기 
prompt_rag_from_hub = client.pull_prompt("prompt_template_rag")
print('\n', "-" * 50, '\n')
print("허브에서 프롬프트를 성공적으로 가져왔습니다.")

# 최근 커밋 해시 조회하기
prompt_id = "prompt_template_rag"
prompt_info_rag = client.get_prompt(prompt_id)

print("프롬프트 정보:", prompt_info_rag)

prompt_info_rag = client.get_prompt(prompt_id)

print("프롬프트 정보:")
for key, value in prompt_info_rag.__dict__.items():
    print(f"{key}: {value}")

<small>

* 셀 출력

    ```<plaintext>
    -------------------------------------------------- 

    ✅ 프롬프트 업로드 결과: https://smith.langchain.com/prompts/prompt_template_rag/cbfb554c?organizationId=2c3342d3-1170-4ffa-86fd-f621199e0b9c

    --------------------------------------------------

    -------------------------------------------------- 

    허브에서 프롬프트를 성공적으로 가져왔습니다.
    프롬프트 정보: repo_handle='prompt_template_rag' description='' readme='' id='9c7f28f1-bf38-4660-91df-bb615a9def9c' tenant_id='2c3342d3-1170-4ffa-86fd-f621199e0b9c' created_at=datetime.datetime(2025, 8, 14, 3, 32, 59, 326434) updated_at=datetime.datetime(2025, 8, 14, 3, 33, 1, 728111) is_public=False is_archived=False tags=['ChatPromptTemplate'] original_repo_id=None upstream_repo_id=None owner=None full_name='prompt_template_rag' num_likes=0 num_downloads=1 num_views=0 liked_by_auth_user=False last_commit_hash='cbfb554c1f4cc2909361ddb0e9ba57709576a63e85e09c760acca822bd5adf21' num_commits=1 original_repo_full_name=None upstream_repo_full_name=None
    프롬프트 정보:
    repo_handle: prompt_template_rag
    description: 
    readme: 
    id: 9c7f28f1-bf38-4660-91df-bb615a9def9c
    tenant_id: 2c3342d3-1170-4ffa-86fd-f621199e0b9c
    created_at: 2025-08-14 03:32:59.326434
    updated_at: 2025-08-14 03:33:01.728111
    is_public: False
    is_archived: False
    tags: ['ChatPromptTemplate']
    original_repo_id: None
    upstream_repo_id: None
    owner: None
    full_name: prompt_template_rag
    num_likes: 0
    num_downloads: 1
    num_views: 0
    liked_by_auth_user: False
    last_commit_hash: cbfb554c1f4cc2909361ddb0e9ba57709576a63e85e09c760acca822bd5adf21
    num_commits: 1
    original_repo_full_name: None
    upstream_repo_full_name: None
    ```

In [ ]:
# gemini로 LLM 초기화 및 Chain 생성하기

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = gemini_lc               # 위에서 정의함

# chain_map 구성
chain_rag = prompt_rag | gemini_lc | StrOutputParser()

# context_text 입력하기
# 간단한 예시
question_text = "머신러닝과 딥러닝의 차이점은 무엇인가요?"
context_text = """머신러닝은 데이터를 기반으로 모델을 학습시켜 예측이나 분류를 수행하는 인공지능의 한 분야입니다. 
딥러닝은 머신러닝의 하위 분야로, 인공신경망(특히 심층 신경망)을 사용하여 데이터를 학습합니다."""

# {question}, {context}
# invoke() 안에도 dict 형태로 넣기
result = chain_rag.invoke({"question": question_text, "context": context_text})
print("\n=== AI 응답 ===\n")
print(result)

<small>

* 셀 출력 (temperatue=0.7, max_tokens=4,096) (1.3s)

    ```<plaintext>
    === AI 응답 ===

    머신러닝은 데이터를 기반으로 모델을 학습시켜 예측이나 분류를 수행하는 인공지능의 한 분야이며, 딥러닝은 머신러닝의 하위 분야로 인공신경망(특히 심층 신경망)을 사용하여 데이터를 학습합니다.
    ```

In [ ]:
# context_text 입력하기
# 다른 예시
# 관련이 있는 질문
question_text_relevant = "서울시가 도입하는 지능형 교통 시스템의 핵심 기능은 무엇인가요?"
context_text = """
최근 서울시는 대중교통 이용 편의성 증진을 위해 '지능형 교통 시스템(ITS)'을 전면 도입한다고 발표했습니다. 이 시스템의 핵심은 빅데이터 분석을 기반으로 실시간 교통 흐름을 예측하고, 버스 도착 정보를 더 정확하게 제공하는 것입니다. 이와 함께, 시범 운영 중인 자율주행 셔틀은 서울 도심의 특정 노선을 운행하며 시민들의 호응을 얻고 있습니다. 이 셔틀은 5G 통신 기술과 고정밀 지도를 활용하여 안전성을 높였으며, 향후 서울 전역으로 확대될 예정입니다. 또한, 서울시는 교통약자를 위한 '스마트 휠체어' 서비스를 도입하여, 전용 앱으로 휠체어를 대여하고 반납할 수 있는 시스템을 구축했습니다. 이 모든 기술은 시민들의 이동 경험을 혁신하고, 미래 도시로의 전환을 가속화할 것으로 기대됩니다.
"""

# {question}, {context}
# invoke() 안에도 dict 형태로 넣기
result = chain_rag.invoke({"question": question_text_relevant, "context": context_text})
print("\n=== AI 응답 ===\n")
print(result)

<small>

* 셀 출력 (temperatue=0.7, max_tokens=4,096) (0.8s)

    ```<plaintext>
    === AI 응답 ===

    서울시가 도입하는 지능형 교통 시스템(ITS)의 핵심 기능은 빅데이터 분석을 기반으로 실시간 교통 흐름을 예측하고, 버스 도착 정보를 더 정확하게 제공하는 것입니다.
    ```

In [ ]:
# 관련 없는 질문 
question_text_irrelevant = "자율주행 셔틀이 도입될 첫 번째 해외 도시는 어디인가요?"

# {question}, {context}
# invoke() 안에도 dict 형태로 넣기
result = chain_rag.invoke({"question": question_text_irrelevant, "context": context_text})
print("\n=== AI 응답 ===\n")
print(result)

<small>

* 셀 출력 (temperatue=0.7, max_tokens=4,096) (0.7s)

    ```<plaintext>
    === AI 응답 ===

    주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.
    ```

In [ ]:
# 조금 더 긴 context_text로 테스트
context_text = """
최근 이집트 카이로 남부의 사카라 지역에서 고대 무덤과 여러 부장품이 발굴되어 학계의 이목을 집중시키고 있습니다. 이 무덤은 약 2,500년 전의 것으로 추정되며, 발굴팀은 이곳에서 정교하게 채색된 나무 관(사르코파구스) 50여 개와 수십 개의 미라를 발견했습니다. 특히, 이 중 한 관에서는 '호루스의 눈' 문양이 새겨진 황금 가면이 출토되어 고대 이집트의 예술성과 종교적 신념을 엿볼 수 있게 합니다.

또한, 이 발굴 현장에서는 희귀한 파피루스 두루마리가 발견되었는데, 여기에는 당시 사용되던 미지의 상형문자가 기록되어 있어 고고학자들의 해독 노력이 이어지고 있습니다. 이 파피루스는 당시 사람들의 일상생활과 종교 의식에 대한 새로운 단서를 제공할 것으로 기대됩니다. 발굴 책임자인 마리암 박사는 "이번 발견은 사카라 지역의 역사를 다시 쓸 만큼 중요한 사건"이라며 "유물의 보존 상태가 매우 양호하여 복원 작업이 활발히 진행될 것"이라고 밝혔습니다. 복원된 유물들은 추후 카이로 박물관에 전시될 예정입니다.
"""

# 관련이 있는 질문
question_text_relevant = "이집트 사카라 지역에서 발굴된 유물의 보존 상태는 어떤가요?"

# {question}, {context}
# invoke() 안에도 dict 형태로 넣기
result = chain_rag.invoke({"question": question_text_relevant, "context": context_text})
print("\n=== AI 응답 ===\n")
print(result)

<small>

* 셀 출력 (temperatue=0.7, max_tokens=4,096) (0.7s)

    ```<plaintext>
    === AI 응답 ===

    발굴된 유물의 보존 상태가 매우 양호하여 복원 작업이 활발히 진행될 것입니다.
    ```

In [ ]:
# 관련 없는 질문 
question_text_irrelevant = "마리암 박사의 전공 분야는 무엇인가요?"

# {question}, {context}
# invoke() 안에도 dict 형태로 넣기
result = chain_rag.invoke({"question": question_text_irrelevant, "context": context_text})
print("\n=== AI 응답 ===\n")
print(result)

<small>

* 셀 출력 (temperatue=0.7, max_tokens=4,096) (0.9s)

    ```<plaintext>
    === AI 응답 ===

    마리암 박사의 전공 분야에 대한 정보는 주어진 문맥에서 찾을 수 없습니다.
    ```

---